In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

In [10]:
def get_buy_webpages(city, 
                     state, 
                     pg_num, 
                     htype=[
                         'house',
                         'multi-family',
                     ]):
    
    overhead = 'https://trulia.com'
    dangle = 'for_sale'
    city = city.title()\
               .replace(' ', '_')
    state = state.upper()
    
    dict_alias = {
        'house': 'SINGLE-FAMILY_HOME',
        'condo': 'APARTMENT,CONDO,COOP',
        'townhouse': 'TOWNHOUSE',
        'multi-family': 'MULTI-FAMILY',
        'land': 'LOT%7CLAND',
        'mobile/manufactured': 'MOBILE%7CMANUFACTURED',
        'other': 'UNKNOWN',
    }
    
    aliases = [dict_alias[h] for h in htype]
    houses = ','.join(aliases)
    
    webpage = f'{overhead}/{dangle}/{city},{state}/{houses}_type/{pg_num}_p/'
    return webpage

In [13]:
get_buy_webpages('philadelphia', 'pa', 1)

'https://trulia.com/for_sale/Philadelphia,PA/SINGLE-FAMILY_HOME,MULTI-FAMILY_type/1_p/'